## Import Packages

In [10]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.tree import plot_tree
from sklearn.tree.export import export_text
from sklearn import preprocessing

## Load in Data

In [2]:
data = pd.read_csv("cobra-clean.csv", sep=',', header=0, usecols = lambda column : column in ["Occur Time", "Latitude", "Longitude"])
data.head()

,Occur Time,Longitude,Latitude
0,1145,-84.49328,33.68845
1,1330,-84.39201,33.75320
2,1450,-84.36212,33.84676
3,1600,-84.49773,33.68677
4,1645,-84.34875,33.75786


In [20]:
le = preprocessing.LabelEncoder()
# Read the data for the training features
# dataX = pd.read_csv("cobra-clean.csv", sep=',', header=0, usecols = lambda column : column in ["Neighborhood", "Month", "Day", "Shift"])
dataX = pd.read_csv("data/examples/cobra_example_10.csv", sep=',', header=0, usecols = lambda column : column in ["Neighborhood", "Month", "Day", "Shift"])
le.fit(dataX["Neighborhood"])
# print(list(le.classes_))
# print(le.transform(dataX["Neighborhood"]))
dataX["Neighborhood"] = le.transform(dataX["Neighborhood"])
le.fit(dataX["Shift"])
# print(list(le.classes_))
# print(le.transform(dataX["Neighborhood"]))
dataX["Shift"] = le.transform(dataX["Shift"])
X = dataX
X.head()

,Neighborhood,Month,Day,Shift
0,2,6,2,0
1,0,1,7,2
2,3,8,15,0
3,2,4,17,1
4,1,8,23,1


In [8]:
# Read the data for the training label
# dataY = pd.read_csv("cobra-clean.csv", sep=',', header=0, engine='python', usecols = lambda column : column in ["Crime Score"])
dataY = pd.read_csv("data/examples/cobra_example_10.csv", sep=',', header=0, usecols = lambda column : column in ["Crime Score"])
Y = dataY
Y.head()

,Crime Score
0,2
1,7
2,10
3,7
4,4


In [21]:
# Train
clf = DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [6]:
# Read the data for the test features
dataZ = pd.read_csv("cobra-2019.csv", sep=',', header=0, engine='python', usecols = lambda column : column in ["Neighborhood", "Month", "Day", "Shift"])
dataZ.head()
le.fit(dataZ["Neighborhood"])
# print(list(le.classes_))
# print(le.transform(dataZ["Neighborhood"]))
dataZ["Neighborhood"] = le.transform(dataZ["Neighborhood"])
le.fit(dataZ["Shift"])
# print(list(le.classes_))
# print(le.transform(dataZ["Neighborhood"]))
dataZ["Shift"] = le.transform(dataZ["Shift"])
Z = dataZ
# Read the data for the test label
dataW = pd.read_csv("cobra-2019.csv", sep=',', header=0, engine='python', usecols = lambda column : column in ["Crime Score"])
dataW.head()
W = dataW

In [ ]:
# Predict based on the given features for the test set
predictions = clf.predict(Z)
predictions_probability = clf.predict_proba(Z)
# Compare to the actual labels
'''Insert comparison methods here'''
# Plot the tree
plot_tree(clf.fit(X, iris.target))

# Export a text file containing the rules for the decision tree
'''decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2) # 
decision_tree = decision_tree.fit(X, Y)
r = export_text(decision_tree, feature_names=iris['feature_names'])
print(r)'''